# Обучение головы (FFN) на эмбеддингах

Если запускаете, полностью клонировав репозиторий локально

In [2]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

Если используете блокнот независимо от проекта

In [ ]:
! pip install --upgrade git+https://github.com/rimgro/biocadprotein.git
clear_output()

In [7]:
import numpy as np

from fpgen.prop_prediction.dataset import FPbase
from fpgen.prop_prediction.metrics import (
    get_regression_metrics,
    get_classification_metrics
)

In [24]:
def preproc(line):
    clean_line = line.replace('\n', ' ').strip('[]')
    numbers = np.fromstring(clean_line, sep=' ')

    return numbers

In [31]:
dataset = FPbase('../data/dataset_embedd.csv', preprocess_function=preproc)
dataset.to_train_dataframe().head()

,sequence,brightness,em_max,ex_max,ext_coeff,lifetime,maturation,pka,stokes_shift,qy,agg,switch_type
558,"[-0.00530367903, 0.00854283385, 0.00128412445,...",-0.516789,-1.357357,-1.875798,-0.814071,NaN,NaN,0.323540,0.923046,-0.056729,m,b
149,"[-0.00167965586, 0.00413581124, 0.00335621182,...",-0.802832,-0.408006,-0.214689,-1.192834,NaN,NaN,1.878962,-0.403015,-0.465539,NaN,b
184,"[-0.00447917636, -0.00560046919, 0.00733367307...",-1.040228,0.883734,0.758032,-0.257845,NaN,NaN,NaN,0.074367,-1.725418,NaN,b
291,"[-0.00244280021, 0.0043460303, 0.00649108412, ...",NaN,-0.516948,-0.184759,NaN,NaN,NaN,NaN,-0.641706,NaN,m,b
30,"[-0.00111842179, -0.00419556629, 0.00596778374...",NaN,-0.610327,-0.244619,NaN,NaN,NaN,NaN,-0.694749,NaN,t,b
